# Kaggle Titanic Survival Competition

In [195]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Understanding the dataset

First an foremost I want to understand what the dataset that I am working with looks like. A few general questions that I want to answer are:

- What are the different columns? How many are there?
- What does the data look like?
- What data types are in the dataset?
- How many entries are there in the dataset? 

In [196]:
titanic_df = pd.read_csv('./data/train.csv')

print(titanic_df.columns)
print(len(titanic_df.columns))

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
12


In [197]:
titanic_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [198]:
print(titanic_df.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [199]:
num_entries = titanic_df.shape[0]
print("Number of entries:", num_entries)

Number of entries: 891


## Data Cleaning

In the cleaning process I am doing some data checking and manipulation tasks that will hopefully improve the usability and quality of precitions in the later steps. 

These include:
- Possibly dropping unnecessary columns
- Handling missing values by either removing the entries, or adding data
- Type checking & possibly converting them to more suitable types
- Removing duplicate data
- Possibly adapting granularity of the data (either making it less or more granular)
- Finding outliers and handling them appropriately
- Possibly adding new features with feature engineering

In [200]:
# First I create a deep copy to guarantee that the original dataframe stays as it is
t_df = titanic_df.copy(deep=True)

### Checking for duplicates

First we sort the data, then we check for duplicates. It is important to sort the data first, since the duplicated() method only compare each row to the previous row. So if there are rows between duplicates, the duplicated() method will not catch these.

For the sorting values I chose the 'Ticket' and 'Cabin' features.


In [201]:
# Checking all rows from the original DF for duplicates
t_df_sorted = t_df.sort_values(by=['Ticket', 'Cabin'])
duplicates = t_df_sorted.duplicated()
number_duplicate = duplicates.sum()
print("Number of duplicates:",number_duplicate)

Number of duplicates: 0


### Dropping/ removing columns

As each row already has an ID, the PassengerID column is redundant and can be dropped.

In [202]:
# DF with dropped columns
t_df_dropped = t_df.drop(['PassengerId', 'SibSp', 'Parch'], axis=1)

### Handling missing values

One way of dealing with missing values would be to infer the data from similar entries, rather than removing the entry altogether. For example, suppose we have a passenger with a missing value for cabin:

- We could try to find out if they are married to another passenger by checking the name.
- We could check if the class gives us a rough estimate of where the cabin would be.
- We could check the price to infer what type of cabin it might be and where it would be located.

Whilst this may sound sensible, it is by no means a hard and fast rule and could therefore make the model perform worse. It is therefore a good idea to split the datasets into two versions: one where all the entries with missing values are removed, and one where the missing values are inferred from other features.

In [203]:
# DF with removed missing values 
t_df_removed = t_df.copy(deep=True)
t_df_removed = t_df.dropna()

# DF with removed missing values & dropped columns
t_df_dropped_removed = t_df_dropped.copy(deep=True)
t_df_dropped_removed = t_df_dropped.dropna()

t_df_dropped_removed

,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C85,C
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,C123,S
6,0,1,"McCarthy, Mr. Timothy J",male,54.0,17463,51.8625,E46,S
10,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,PP 9549,16.7000,G6,S
11,1,1,"Bonnell, Miss. Elizabeth",female,58.0,113783,26.5500,C103,S
...,...,...,...,...,...,...,...,...,...
871,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,11751,52.5542,D35,S
872,0,1,"Carlsson, Mr. Frans Olof",male,33.0,695,5.0000,B51 B53 B55,S
879,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,11767,83.1583,C50,C
887,1,1,"Graham, Miss. Margaret Edith",female,19.0,112053,30.0000,B42,S


Here we can see that removing the rows with missing values results in a dataframe with only 183 rows compared to the original 891. The significantly reduced dataset might affect the quality of the model, it might be therefore a better approach to add missing values instead of removing them.

### Checking and converting the types

To male operations more effictient it is recommended effectively convert the types to the smallest possible data types. For example the 'Survived' feature is comprised of only 0 and 1, therefore it does not make sense for it to be an INT64 data type. A much more efficient type would be the boolean data type. 

In [208]:
# Type conversion of DF with removed values
t_df_removed.loc[:, 'Pclass'] = t_df_removed['Pclass'].astype('int8')
t_df_removed.loc[:, 'Age'] = t_df_removed['Age'].astype('int8')
t_df_removed.loc[:, 'Parch'] = t_df_removed['Parch'].astype('int8')
t_df_removed.loc[:, 'Survived'] = t_df_removed['Survived'].astype('bool')
t_df_removed[['Pclass', 'Age', 'Survived', 'Parch', 'SibSp']] = t_df_removed[['Pclass', 'Age', 'Survived','Parch', 'SibSp']].astype({'Pclass': 'int8', 'Age': 'int8', 'Survived': 'bool', 'Parch':'int8', 'SibSp':'int8'})

print(t_df_removed.dtypes)

# Type conversion of DF with removed values & dropped columns
t_df_dropped_removed[['Pclass', 'Age', 'Survived']] = t_df_dropped_removed[['Pclass', 'Age', 'Survived']].astype({'Pclass': 'int8', 'Age': 'int8', 'Survived': 'bool'})

print(t_df_dropped_removed.dtypes)



PassengerId      int64
Survived          bool
Pclass            int8
Name            object
Sex             object
Age               int8
SibSp             int8
Parch             int8
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
Survived       bool
Pclass         int8
Name         object
Sex          object
Age            int8
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object


/var/folders/6d/y3v3tvc170j90w1hsnzm4jmr0000gn/T/ipykernel_70869/790069340.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_df_removed[['Pclass', 'Age', 'Survived', 'Parch', 'SibSp']] = t_df_removed[['Pclass', 'Age', 'Survived','Parch', 'SibSp']].astype({'Pclass': 'int8', 'Age': 'int8', 'Survived': 'bool', 'Parch':'int8', 'SibSp':'int8'})
/var/folders/6d/y3v3tvc170j90w1hsnzm4jmr0000gn/T/ipykernel_70869/790069340.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_df_dropped_removed[['Pclass', 'Age',

### Adapting the granularity 

In the original dataset the features 'SibSp' and 'Parch' are aggregates of information:
- SibSp refers to the number of siblings / spouses aboard the Titanic
- Parch refers to the number of parents / children aboard the Titanic

Even though we are not yet sure if we are going to need the data, it might be worthwhile to split up the data in the following, more granular way:

- Number of Siblings on board
- Number of Spouses on board
- Number of Parents on board
- Number of Children on board

To achieve this we need to find a way to appropriately split up the data. 

To find out if the value in 'Parch' refers to either children or parents an approach could be to check the age. If the age is below 16 it is unlikely that the value in Parch is referring to the number of children of that passenger. Please note that this is not a very precise way of handling the data. It needs to be checked latter, if this does not make the predictions less reliable.  

In [205]:
# Checking the ages of all passengers from the original DF
age_counts = t_df['Age'].value_counts().sort_index()
print(age_counts)

Age
0.42     1
0.67     1
0.75     2
0.83     2
0.92     1
        ..
70.00    2
70.50    1
71.00    2
74.00    1
80.00    1
Name: count, Length: 88, dtype: int64


## Definition of the questions to be answered

The overall question is: What are the main factors that contributed to the survival rate of a passenger on the Titanic?

### Defining the assumptions to test

To build a successful model that attempts to predict survivability, it is first necessary to find related features. To find possible features that affect survivability, I first define some assumptions that I want to test:

- Age has an effect on survivability (due to the fitness of the individual)
- The cabin has an effect on survivability (due to proximity to exits, etc.)
- Class has an effect on survivability (due to the fact that higher class passengers had priority on emergency ships)
- Tickets and cabins are a proxy for class and therefore affect survivability (they reflect the class of passenger).

Please note that these assumptions are completely made up and based on intuition. They are just there to give me something to work with and investigate. Even if there is a relationship between survivability and the feature, this does not necessarily mean that there is a correlation or proof of my initial assumption.